In [62]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

## Preparing data in the format for ner tags

Original input data is in format :
* Each input article was given in seperated text file (article id in text file name), for train and dev sets
* With the same article id in label file , we get annotation of propaganda spans:
    * article_id propaganda start_span end_span

    * article id 1111 is present in article_1111.txt and label is present in article_1111_TC.txt 
         * article_1111.txt : Trump the white president say he likes black people working from him .
         * article_1111_TC.txt : 
             * 1111 <propaganda type> 11 88

    
From this input we convert we covert the articles to files for each article ids , but in positions of span text , markings of start span and end span.This is done by using the scripts given by the competition organizer. For example : 

 * the output after processing looks like 
    
    article_1111.txt : Trump the <span-7> white president say he likes black people working from him. <7-/span> 
    
    the id 7 comes from type of propaganda. each propaganda has been given a seperated id .
    
From the above output the following preprocessing turns this article id into seperate sentences each having ner like labels
    
Example 
    
      file no : article_1111.txt
      
      Trump O
      the O
      white I 
      president I
      say I
      he I
      likes I
      black I
      people I
      working I
      from I
      him I
      . I


In [63]:
import pandas as pd
import glob
import swifter
path = r'/data/semeval-2020/task-11/datasets/train-tagged_article/'
all_files = glob.glob(path + "/*.txt")

content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    content_tuple.append((article_id,content))

In [64]:
content_df = pd.DataFrame(content_tuple,columns=["article_id","content"])

In [65]:
content_df.head(3)

article_id  \
0  762956953   
1  787529309   
2  999001296   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [66]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English   # updated , 
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
tokenizer = Tokenizer(nlp.vocab)

content_df.content[40]

'Democrat’s Friend Louis Farrakhan In Iran: “<span-11 <span-8 Death to America8-/span>!” – America Is The “Great Satan 11-/span>”  Ah yes, Nation of Islam leader, racist, and totalitarian Louis Farrakhan was recently captured on video in Tehran, Iran chanting “<span-10 <span-11 <span-8 Death to America11-/span>8-/span>” 10-/span> in Arabic and stating that America is the “Great Satan.” Louis Farrakhan, leader of Nation of Islam, met with Mohsen Rezaee, secretary of Iran‘s Expediency Discernment Council, on Sunday in Tehran. Heres’ what he had to say. Iran state TV news presenter: Listen to leader of Nation of Islam chanting "<span-11 <span-10 <span-8 Death to America11-/span>10-/span> 8-/span>" Farrakhan [in Farsi]: "Death to…" Audience: "…America" pic.twitter.com/p0qGfHHfg0 — Sobhan Hassanvand (@Hassanvand) November 4, 2018 He then went on in a university speech to encourage the revolution and to persevere against the sanctions of the US, whom he referred to as the “Great Satan.” Loui

In [67]:
for i in nlp(content_df.content[40]).sents:
    print(i)

Democrat’s Friend Louis Farrakhan In Iran: “<span-11 <span-8 Death to America8-/span>!” –
America Is The “Great Satan 11-/span>”  Ah yes, Nation of Islam leader, racist, and totalitarian Louis Farrakhan was recently captured on video in Tehran, Iran chanting “<span-10 <span-11 <span-8 Death to America11-/span>8-/span>” 10-/span> in Arabic and stating that America is the “Great Satan.”
Louis Farrakhan, leader of Nation of Islam, met with Mohsen Rezaee, secretary of Iran‘s Expediency Discernment Council, on Sunday in Tehran.
Heres’ what he had to say.
Iran state TV news presenter: Listen to leader of Nation of Islam chanting "<span-11 <span-10 <span-8 Death to America11-/span>10-/span> 8-/span>" Farrakhan [in Farsi]: "Death to…" Audience: "…America" pic.twitter.com/p0qGfHHfg0 — Sobhan Hassanvand (@Hassanvand) November 4, 2018 He then went on in a university speech to encourage the revolution and to persevere against the sanctions of the US, whom he referred to as the “Great Satan.”
Louis

In [68]:
def split_into_sentences(row):
    sentences = []
    for ix,sent in enumerate(nlp(row.content).sents):
        sentences.append((row.article_id,ix,sent.text))
    return sentences    

In [69]:
sentences_dataset = content_df.apply(split_into_sentences,axis=1).values.tolist()

In [70]:
len(sentences_dataset)

371

In [71]:
sentences_dataset = [j for i in sentences_dataset for j in i]

In [72]:
len(sentences_dataset)

15565

In [73]:
sentences_dataset[:10]

[('762956953',
  0,
  'Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks  This has long been known, although the mainstream media dismissed it as a conspiracy theory.'),
 ('762956953', 1, 'But now we have definitive confirmation.'),
 ('762956953',
  2,
  'It was Iran Bush should have invaded after 9/11 , not Iraq.'),
 ('762956953',
  3,
  'Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.'),
 ('762956953',
  4,
  'The Iran nuclear deal should never have proceeded — President Obama, <span-6 the worst president in American history 6-/span> . “'),
 ('762956953',
  5,
  'Iran Admits To Facilitating 9/11 Terror Attacks,” by Adam Kredo, Washington Free Beacon, June 8, 2018: Iranian officials, in a first, have admitted to facilitat

In [74]:
sentences_df = pd.DataFrame(sentences_dataset,columns=["article_id","sentence_id","sentence"])

In [75]:
sentences_df.head(4)

,article_id,sentence_id,sentence
0,762956953,0,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory."
1,762956953,1,But now we have definitive confirmation.
2,762956953,2,"It was Iran Bush should have invaded after 9/11 , not Iraq."
3,762956953,3,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic."


In [82]:
"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory.".find("span")

-1

In [83]:
def has_propaganda(sentence):
    if sentence.find("span") > 0:
        return True
    return False

In [84]:
sentences_df["has_propaganda"] = sentences_df["sentence"].apply(has_propaganda)

In [85]:
sentences_df.head(4)

,article_id,sentence_id,sentence,has_propaganda
0,762956953,0,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory.",False
1,762956953,1,But now we have definitive confirmation.,False
2,762956953,2,"It was Iran Bush should have invaded after 9/11 , not Iraq.",False
3,762956953,3,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.",False


In [86]:
sentences_df.has_propaganda.value_counts()

False    10596
True     4969 
Name: has_propaganda, dtype: int64

In [89]:
import re
def cleaned_sentence(sentence):
    return re.sub("<span-\d+", "", re.sub("\d+-/span>", "", sentence))

In [90]:
sentences_df["cleaned_sentence"] = sentences_df["sentence"].apply(cleaned_sentence)

In [92]:
sentences_df.head()

,article_id,sentence_id,sentence,has_propaganda,cleaned_sentence
0,762956953,0,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory.",False,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory."
1,762956953,1,But now we have definitive confirmation.,False,But now we have definitive confirmation.
2,762956953,2,"It was Iran Bush should have invaded after 9/11 , not Iraq.",False,"It was Iran Bush should have invaded after 9/11 , not Iraq."
3,762956953,3,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.",False,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic."
4,762956953,4,"The Iran nuclear deal should never have proceeded — President Obama, <span-6 the worst president in American history 6-/span> . “",True,"The Iran nuclear deal should never have proceeded — President Obama, the worst president in American history . “"
